# Задача 1. Линейная регрессия, метод градиентного спуска

## Постановка задачи

- [ ] Самостоятельно реализовать функцию **gradient_descent(X, y)**, которая по заданной обучающей выборке обучает модель линейной регрессии, оптимизируя функционал методом градиентного спуска (Batch Gradient Descent, GD) и возвращая вектор весов **w**. В качестве функционала можно выбрать, например, функцию ошибок **MSE** + $L_2$-регуляризатор. Использовать матрично-векторные операции для вычисления градиента.
- [ ] Найти данные, на которых интересно будет решать задачу регрессии. Зависимость целового признака от нецелевых должна быть не слишком сложной, чтобы обученная линейная модель смогла показать приемлимый результат. В крайнем случае взять данные для предсказания стоимости машин [тут](https://github.com/rustam-azimov/ml-course/tree/main/data/car_price) (целевой признак для предсказания --- **price**).
- [ ] Считать данные, выполнить первичный анализ данных, при необходимости произвести чистку данных (Data Cleaning).
- [ ] Выполнить разведочный анализ (EDA), использовать визуализацию, сделать выводы, которые могут быть полезны при дальнейшем решении задачи регрессии.
- [ ] При необходимости выполнить полезные преобразования данных (например, трансформировать категариальные признаки в количественные), убрать ненужные признаки, создать новые (Feature Engineering).
- [ ] Случайным образом разбить данные на обучающую и тестовую выборки, используя методы существующих библиотек.
- [ ] При обучении моделей использовать масштабирование данных.
- [ ] Обучить модель на обучающей выборке, используя функцию **gradient_descent(X, y)**. Оценить качество модели на обучающей и тестовой выборках, используя **MSE**, **RMSE** и $R^2$.
- [ ] Обучить модель, используя существующую библиотеку. Например, в **sklearn** для $L_2$-регуляризатора можно использовать **Ridge**. Сравнить качество с вашей реализацией.
- [ ] Повторить тоже самое, но используя кросс-валидацию.
- [ ] Создать таблицу, со строками (mse-train, mse-test, rmse-train, rmse-test, r2-train, r2-test) и столбцами (Fold1, Fold2, ..., Foldk, E, STD), где k --- количество фолдов в кросс-валидации, E --- мат. ожидание и STD --- стандартное отклонение. Сделать выводы.
- [ ] * (+1 балл) Перед обучением моделей подобрать наилучшее количество (и само подмножество) признаков, например используя Recursive Feature Elimination (RFE).
- [ ] * (+1 балл) Во все ваши реализации добавить возможность настройки нужных гиперпараметров, а в процессе обучения **всех** моделей осуществить подбор оптимальных значений этих гиперпараметров.
- [ ] * (+2 балла) Также самостоятельно реализовать метод стохастического градиентного спуска (Stochastic Gradient Descent, SGD), обучить модели и добавить их во все сравнения.
- [ ] * (+2 балла) Также самостоятельно реализовать метод мини-пакетного градиентного спуска (Mini Batch Gradient Descent), обучить модели и добавить их во все сравнения.

In [1]:
import numpy as np
from numpy.core.records import ndarray

# Реализация линейной модели

In [4]:
def get_full_sample_matrix(samples: ndarray):
    samples_matrix = samples.copy()
    if samples.ndim == 1:
        samples_matrix = samples_matrix.reshape(-1, 1)

    ones_vec = np.ones((samples_matrix.shape[0], 1), dtype=samples.dtype)
    return np.hstack([ones_vec, samples_matrix])

def linear_model(w: ndarray, x: ndarray) -> ndarray:
    return np.dot(get_full_sample_matrix(x), w)

20

# Реализация градиентного спуска

In [ ]:
def gradient_descent(X, y, w0, epsilon=1E-8, alpha=1, lmda=0.001, dw=1E-8):

    def cost(w, **kwargs):
        x = kwargs['x']
        y_sample = kwargs['y']
        alpha_reg = kwargs['alpha_reg']

        y_new = linear_model(w, x)

        c = sum((y_sample - y_new) ** 2) / len(y_sample)
        # L2 регуляризация
        c += alpha_reg * sum(w ** 2)
        return c

    run_condition = True

    cost_inputs = {
        'x': X,
        'y': y,
        'alpha_reg': alpha
    }
    costs = [cost(w0, **cost_inputs)]
    cur_w = w0

    while run_condition:
        # считаем градиент
        del_c = []
        for i, w_i in enumerate(w0):
            tw = cur_w
            tw[i] = w_i + dw
            cost_i = cost(tw, cost_inputs)
            dc = cost_i - costs[-1]
            del_c.append(dc)

        cur_w = cur_w - lmda * np.array(del_c)

        costs.append(cost(cur_w, **cost_inputs))

        run_condition = abs(costs[-1] - costs[-2]) > epsilon

    return cur_w

# Датасет